卷积神经网络（Convolutional Neural Network，CNN）

相邻层的所有神经元之间都有连接，这称为全连接（fully connected），在全连接的神经网络中，Affine 层后面跟着激活函数ReLu层或者Sigmoid层。
而CNN中新增了Convolution层和Pooling层。CNN的层的连接顺序是“Convolution - ReLU -（Pooling）”（Pooling层有时会被省略）。这可以理解为之前的“Afne - ReLU”连接被替换成了“Convolution - ReLU -（Pooling）”连接。靠近输出的层中使用了之前的“Afne - ReLU”组合。此外，最后的输出层中使用了之前的“Afne - Softmax”组合。这些都是一般的CNN中比较常见的结构。

全连接层存在的问题：

    之前介绍的全连接的神经网络中使用了全连接层（Affine层）。在全连接层中，相邻的神经元全部链接在一起，输出的数量可以任意决定。

    但是，这种全连接的结构有一个问题。就是数据的形状被忽视了，图像数据是三维数据（高、长、深），而全连接层的输入是需要将3维数据拉平为1维数据。

    为了解决这个问题，卷积神经网络（Convolutional Neural Network，CNN）被提出了。

    图象是3维形状，这个形状中应该含有重要的空间信息。比如，空间上邻近的像素为相似的值、RBG的各个通道之间分别有密切的关联性、相距较远的像素之间没有什么关联等，3维形状中可能隐藏有值得提取的本质模式。但是，因为全连接层会忽视形状，将全部的输入数据作为相同的神经元（同一维度的神经元）处理，所以无法利用与形状相关的信息，而卷积层可以保持形状不变，当输入数据是图像时，卷积层会以3维数据的形式接收输入数据，并同样以3维数据的形式输出至下一层，因此，在CNN中，可以（有可能）正确理解图像等具有形状的数据。

    另外，CNN中，有时将卷积层的输入输出数据称为特征图（feature map）。其中，卷积层的输入数据称为输入特征图（input feature map），输出数据称为输出特征图（output feature map）。

    

#### 卷积运算

对于输入数据，卷积运算以一定间隔滑动卷积核，将卷积核中的元素和输入数据中的元素一一对应相乘，并将乘积相加，得到输出数据中的一个元素。这里，乘积和相加的运算称为元素wise乘法（elementwise multiplication）和加法（addition）。

在全连接的神经网络中，除了权重参数，还存在偏置。CNN中，滤波器的参数就对应之前的权重。并且，CNN中也存在偏置。偏置通常只有1个（1×1），这个值和卷积后的每个元素相加作为输出数据

#### 填充

在进行卷积层的处理之前，有时要向输入数据的周围填入固定的数据（比如0等），这称为填充（padding），是卷积运算中经常会用到的处理。使用填充主要是为了调整输出的大小。比如，对大小为(4,4)的输入数据应用(3,3)的滤波器时，输出大小变为(2,2)，相当于输出大小比输入大小缩小了2个元素。这在反复进行多次卷积运算的深度网络中会成为问题。为什么呢？因为如果每次进行卷积运算都会缩小空间，那么在某个时刻输出大小就有可能变为1，导致无法再应用卷积运算。为了避免出现这样的情况，就要使用填充。在刚才的例子中，将填充的幅度设为1，那么相对于输入大小(4,4)，输出大小也保持为原来的(4,4)。因此，卷积运算就可以在保持空间大小不变的情况下将数据传给下一层。


#### 步幅

应用卷积核的位置间隔称为步幅（stride）。增大步幅后，输出大小会变小。而增大填充后，输出大小会变大。
假设输入大小为(H,W)，滤波器大小为(FH,FW)，输出大小为(OH,OW)，填充为P，步幅为S。
输出大小的计算如下式所示。

$$OH=\frac{H+2P-FH}{S}+1$$
$$OW=\frac{W+2P-FW}{S}+1$$

当输出大小无法除尽时（结果是小数时），需要采取报错等对策。根据深度学习的框架的不同，当值无法除尽时，有时会向最接近的整数四舍五入，不进行报错而继续运行。


#### 3维数据的卷积运算

图像是3维数据，除了高、长方向之外，还需要处理通道方向，需要注意的是，在3维数据的卷积运算中，输入数据和卷积核的通道数要设为相同的值，卷积核大小可以自由设定，但是每个通道的卷积核大小要相同。

通道数只能设定为和输入数据的通道数相同的值 ！！！！！！

通道数为C、高度为H、长度为W的数据的形状可以写成（C,H,W）。滤波器也一样，要按（channel, height,width）的顺序书写。比如，通道数为C、滤波器高度为FH（Filter Height）、长度为FW（Filter Width）时，可以写成（C,FH,FW）。

（C,H,W） *  (C,FH,FW) -----> (1,OH,OW)

这样一个卷积核完成对一张图片的卷积运算后，生成的一个输出数据（即一张特征图）形状为（1，OH，OW）。所谓1张特征图，就是通道数为1 的特征图。


要生成多个特征图，需要设定多个卷积核。比如，设定为3个卷积核，就会生成3个特征图。FN个卷积核会生成FN个特征输出图将这FN个特征图汇集在一起，就得到了形状为(FN,OH,OW)的方块。将这个方块传给下一层，就是CNN的处理流。

（C,H,W） *  (FN,C,FH,FW) -----> (FN,OH,OW)


每个通道只有一个偏置，偏置的形状是(FN,1,1)，滤波器的输出结果的形状是(FN,OH,OW)对滤波器的输出结果(FN,OH,OW)按通道加上相同的偏置值

(C, H, W) * (FN, C, FH, FW) ------>  (FN, OH, OW)  +  (FN, 1, 1) ------> (FN, OH, OW)

#### 批处理